In [ ]:
import glob
import pandas as pd
import tqdm
import json
SCENARIOS_DIR = "/shared_workspace_mfs/kirill/code_architect/LoCoBench/data/output/scenarios"

In [ ]:
# Get all the architecture_understanding samples together
arch_q_fnames = []
arch_q_fnames = [fname for fname in glob.glob(f"{SCENARIOS_DIR}/*architectural_understanding*")]

df = pd.DataFrame()
for fname in tqdm.tqdm(arch_q_fnames):
    with open(fname, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    raw = pd.json_normalize(data, sep='_')
    raw['fpath'] = fname

    df = pd.concat([df, raw], ignore_index=True, axis=0)

In [ ]:
# filter only most relevant columns
df = df[['id', 'task_category', 'difficulty', 'title', 'description', 'context_files', 'context_length', 'task_prompt', 'expected_approach', 'ground_truth',
    'evaluation_criteria', 'metadata_context_length', 'metadata_files_count', 'metadata_information_coverage', 'metadata_coverage_range',
    'metadata_generation_timestamp']]

In [ ]:
import requests
import json
from dotenv import load_dotenv
import os

load_dotenv()

def llm_request(sys_prompt, usr_prompt, model="deepseek/deepseek-v3.2"):
    """
    OpenRouter LLM
    """
    # First API call with reasoning
    response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {os.getenv("API_KEY")}",
        "Content-Type": "application/json",
    },
    data=json.dumps({
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": sys_prompt
            },
            {
            "role": "user",
            "content": usr_prompt
            }
        ],
        "reasoning": {"enabled": True}
    })
    )
    
    # Extract the assistant message with reasoning_details
    response = response.json()
    response = response['choices'][0]['message']

    return response

In [ ]:
sys_prompt = "You are a software engineer who is an expert in software architecture and design."

usr_prompt = f"I have a set of software architecture questions, and I would like you to analyze them and generate a taxonomy. The goal is to categorize these questions into relevant themes or topics that reflect different aspects of software architecture. The taxonomy should help identify common categories and subcategories of questions, showing relationships between them."
usr_prompt += f"\nHere are the questions: \n\n{',\n'.join(df['title'].to_list())}"

In [ ]:
resp = llm_request(sys_prompt, usr_prompt)

In [ ]:
from prompts import TAXONOMY_PROMPT

df.iloc[0]

In [ ]:
# save arch understanding questions to jsonl
# df.to_json("/shared_workspace_mfs/kirill/code_architect/LoCoBench/data_analysis/arch_understanding_questions.jsonl", lines=True, orient='records')

#### Plots

In [ ]:
fpath = "/shared_workspace_mfs/kirill/code_architect/LoCoBench/data_analysis/test_data.jsonl"
df = pd.read_json(fpath, lines=True, orient='records')

In [ ]:
import re
# --- preprocessing ---
# 1. remove '1. ' numbers where applicable
df['tax_top_level'] = df['tax_top_level'].apply(lambda x: re.sub(r"\d+\. ", "", x))
df['tax_sub_level'] = df['tax_sub_level'].apply(lambda x: re.sub(r"\d+\.\d ", "", x))

# 2. replace instances of ', ' with ',' (no space), easier for separation later
df['tax_top_level'] = df['tax_top_level'].apply(lambda x: re.sub(r", ", ",", x))
df['tax_sub_level'] = df['tax_sub_level'].apply(lambda x: re.sub(r", ", ",", x))

In [ ]:
df.columns

In [ ]:
# breaking down multiple categories per sample
# df['tax_sub_level'].str.get_dummies(sep=",").sum().to_frame()
df['tax_sub_level'].str.split(",").sum()

In [ ]:
import plotly.express as px
# histogram for finegrained taxonomy
px.histogram(x=df['tax_sub_level'].str.split(",").sum(), labels={"x": "Finegrained category"}, title="LoCoBench Architectural Understanding: Finegrained Taxonomy")

In [ ]:
import plotly.express as px
# histogram for coursegrained taxonomy
px.histogram(x=df['tax_top_level'].str.split(",").sum(), labels={"x": "Course category"}, title="LoCoBench Architectural Understanding: Course Taxonomy")